In [1]:
# just comment this symbolic code out since don't want to run this each time I'm doing analysis

# first compute all the kernel derivatives
import sympy as sym

x = sym.Symbol('x')
xp = sym.Symbol('xp')
L = sym.Symbol('L')
c = sym.Symbol('c')
a = sym.Symbol('a')

kernel = sym.exp(-0.5 / L / L * (x - xp)**2) * (c + x * (a-x) * xp * (a - xp))
print(f"{kernel=}")

dx2_kernel = kernel.diff(x, x)
# dx2_kernel = sym.simplify(dx2_kernel)
dx2_kernel_fact = dx2_kernel / kernel
print(f"{dx2_kernel_fact=}")

dx4_kernel = kernel.diff(x, x, x, x)
# dx4_kernel = sym.simplify(dx4_kernel)
dx4_kernel_fact = dx4_kernel /  kernel
print(f"{dx4_kernel_fact=}")

dx6_kernel = kernel.diff(x, x, x, x, xp, xp)
dx6_kernel_fact = dx6_kernel / kernel
# dx6_kernel = sym.simplify(dx6_kernel)
print(f"{dx6_kernel_fact=}")

dx8_kernel = kernel.diff(x, x, x, x, xp, xp, xp, xp)
dx8_kernel_fact = dx8_kernel / kernel
# dx8_kernel = sym.simplify(dx8_kernel)
print(f"{dx8_kernel_fact=}")

kernel=(c + x*xp*(a - x)*(a - xp))*exp(-0.5*(x - xp)**2/L**2)
dx2_kernel_fact=(-2*xp*(a - xp) + 2.0*xp*(-a + 2*x)*(a - xp)*(x - xp)/L**2 - 1.0*(1 - (x - xp)**2/L**2)*(c + x*xp*(a - x)*(a - xp))/L**2)/(c + x*xp*(a - x)*(a - xp))
dx4_kernel_fact=(12.0*xp*(1 - (x - xp)**2/L**2)*(a - xp) - 4*xp*(3.0 - 1.0*(x - xp)**2/L**2)*(-a + 2*x)*(a - xp)*(x - xp)/L**2 + (c + x*xp*(a - x)*(a - xp))*(3.0 - 6.0*(x - xp)**2/L**2 + 1.0*(x - xp)**4/L**4)/L**2)/(L**2*(c + x*xp*(a - x)*(a - xp)))
dx6_kernel_fact=(-24.0 - 2*x*(a - x)*(3.0 - 6.0*(x - xp)**2/L**2 + 1.0*(x - xp)**4/L**4)/L**2 - 12.0*xp*(1 - (x - xp)**2/L**2)*(a - xp)/L**2 - 24.0*xp*(1 - (x - xp)**2/L**2)*(x - xp)/L**2 - 24.0*xp*(a - xp)/L**2 - 48.0*xp*(x - xp)/L**2 + 24.0*(1 - (x - xp)**2/L**2)*(a - xp)*(x - xp)/L**2 - 8*(3.0 - 1.0*(x - xp)**2/L**2)*(a - 2*x)*(x - xp)/L**2 - 8*(3.0 - 1.0*(x - xp)**2/L**2)*(x*xp - x*(a - xp) - xp*(a - x) + (a - x)*(a - xp))/L**2 + 48.0*(a - xp)*(x - xp)/L**2 + 24.0*(x - xp)**2/L**2 - 2*x*(12.0 - 4.0*(x - xp)**2/L*

In [ ]:
# define the kernel functions
import tensorflow as tf
import numpy as np
from _bounded_kernels import *

Lx = 0.4; Ly = 0.4
a, b = (3, 1)
DTYPE = tf.float32
L_tf = tf.constant(np.array([Lx, Ly]), dtype=DTYPE)
def SE_kernel2d_tf(x, xp, Lvec):
    # x input is N x 1 x 2 array, xp is 1 x M x 2 array
    # xbar is then an N x M x 2 shape array
    xbar = (x - xp) / Lvec
    # output is N x M matrix of kernel matrix
    return tf.exp(-0.5 * tf.reduce_sum(tf.pow(xbar, 2.0), axis=-1))

def bndry_fcn(x, a):
    return x * (a-x)

def kernel2d_tf(x, xp):
    """composite kernel definition"""
    x1 = x[:,:,0]; x2 = x[:,:,1]
    x1p = xp[:,:,0]; x2p = xp[:,:,1]
    # temp = SE_kernel2d_tf(x, xp, L_tf)
    # print(f"{temp.shape=}")
    return SE_kernel2d_tf(x, xp, L_tf) * bndry_fcn(x1, a) * bndry_fcn(x2, b) * bndry_fcn(x1p, a) * bndry_fcn(x2p, b)

def kernel2d_bilapl_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    x1 = x[:,:,0]; x2 = x[:,:,1]
    x1p = xp[:,:,0]; x2p = xp[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]
    x1_in = (x1, x1p, a, Lx)
    x2_in = (x2, x2p, b, Ly)

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    return K * \
        (d4_fact(*x1_in) + 2.0 * d2_fact(*x1_in) * d2_fact(*x2_in) + d4_fact(*x2_in))

def kernel2d_double_bilapl_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    x1 = x[:,:,0]; x2 = x[:,:,1]
    x1p = xp[:,:,0]; x2p = xp[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]
    x1_in = (x1, x1p, a, Lx)
    x2_in = (x2, x2p, b, Ly)

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    return K * (d8_fact(*x1_in) + \
                4.0 * d6_fact(*x1_in) * d2_fact(*x2_in) +\
                6.0 * d4_fact(*x1_in) * d4_fact(*x2_in) +\
                4.0 * d2_fact(*x1_in) * d6_fact(*x2_in) +\
                d8_fact(*x2_in))

In [5]:
# verify the kernel function derivatives with tensorflow gradient taping..
import numpy as np
import tensorflow as tf
xy = np.random.rand(1,1,2).astype(np.float32)
x = tf.constant(xy[:,:,0:1]); y = tf.constant(xy[:,:,1:2])
# temp = tf.concat([x, y], axis=0)
xyp = tf.constant(np.random.rand(1,1,2).astype(np.float32))
xp = tf.constant(xyp[:, :, 0:1]); yp = tf.constant(xyp[:, :, 1:2])

with tf.GradientTape(persistent=True) as tape4:
    tape4.watch(x)
    tape4.watch(y)
    with tf.GradientTape(persistent=True) as tape3:
        tape3.watch(x)
        tape3.watch(y)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            tape2.watch(y)
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(x)
                tape1.watch(y)
                K = kernel2d_tf(tf.concat([x, y], axis=2), xyp)
                dKdx = tape1.gradient(K, x)
                dKdy = tape1.gradient(K, y)
                # print(f"{K=} {dKdx=} {dKdy=}")
            dx2 = tape2.gradient(dKdx, x)
            dy2 = tape2.gradient(dKdy, y)
            print(f"{dx2=} {dy2=}")
        dx3 = tape3.gradient(dx2, x)
        dxxy = tape3.gradient(dx2, y)
        dy3 = tape3.gradient(dy2, y)
    dx4 = tape4.gradient(dx3, x)
    dx2y2 = tape4.gradient(dxxy, y)
    dy4 = tape4.gradient(dy3, y)

del tape1, tape2, tape3, tape4

# the nabla^4_x k_x for 2d kernel seems to be correct!
tf_bilapl_x = (dx4 + 2.0 * dx2y2 + dy4).numpy()[0,0,0]
analytic_bilapl_x = kernel2d_bilapl_tf(xy, xyp).numpy()[0,0]
bilapl_rel_err = np.abs((tf_bilapl_x - analytic_bilapl_x) / tf_bilapl_x)
print(f"{tf_bilapl_x=} {analytic_bilapl_x=} {bilapl_rel_err=}")

dx2=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-0.16034347]]], dtype=float32)> dy2=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-0.36691993]]], dtype=float32)>
tf_bilapl_x=16.38039 analytic_bilapl_x=16.38039 bilapl_rel_err=0.0
